IMPORTS

In [ ]:
pip install infomap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.8/264.8 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
import matplotlib as mp
import networkx as nx
import matplotlib.pyplot as plt
import random
import heapq
from collections import defaultdict
from infomap import Infomap

ASSIGNING WEIGHTS TO DATAVALUES

In [ ]:
path = '/content/activitytime.xlsx'
df = pd.read_excel(path)

In [ ]:
df.head()

,Node1,Node2,Timestamp,Type_of_Action
0,223789,213163,1341100972,MT
1,223789,213163,1341100972,RE
2,376989,50329,1341101181,RT
3,26375,168366,1341101183,MT
4,376989,13813,1341101192,RT


In [ ]:
G = nx.DiGraph()

weights = {'RT': 3, 'MT': 2, 'RE': 1}

for index, row in df.iterrows():
    G.add_edge(row['Node2'], row['Node1'], weight=weights.get(row['Type_of_Action'], 1))

print("Sample edges:", list(G.edges())[:5])

Sample edges: [(213163, 223789), (213163, 213163), (213163, 246023), (223789, 213163), (223789, 251638)]


In [ ]:
def score_assign(action):
  weights = {'RT' : 5, 'MT': 3, 'RE' : 1}
  return weights[action]

In [ ]:
#RT = 5
#MT = 3
#RE = 1
from collections import defaultdict

inf_score = defaultdict(int)

for row in df.itertuples(index=False):
    inf_score[row.Node2] += score_assign(row.Type_of_Action)

print(len(dict(inf_score)))


59977


USING INFOMAP TO FORM COMMUNITIES

In [ ]:
im = Infomap()
for u, v, data in G.edges(data=True):
    im.add_link(u, v, data['weight'])
im.run()
communities = []
for node in im.nodes:
    communities.append((node.node_id, node.module_id))
communities_df = pd.DataFrame(communities, columns=["Node", "Community"])
print(communities_df)

          Node  Community
0           88          1
1       215324          1
2       370434          1
3       370435          1
4       171356          1
...        ...        ...
304686  422492      13496
304687  263233      13497
304688  163950      13498
304689  377965      13499
304690  209173      13500

[304691 rows x 2 columns]


In [ ]:
community_counts = communities_df['Community'].value_counts().reset_index()
community_counts.columns = ['Community', 'Node Count']
print(community_counts)


       Community  Node Count
0              1      150141
1              2       44064
2              3       12443
3              4       10139
4              5        8115
...          ...         ...
13495      12656           1
13496      12001           1
13497      13080           1
13498      12659           1
13499      13500           1

[13500 rows x 2 columns]


In [ ]:
average_nodes_per_community = community_counts['Node Count'].mean()

print("Average number of nodes per community:", average_nodes_per_community)


Average number of nodes per community: 22.569703703703702


In [ ]:
threshold = 50 # Replace with your desired threshold

small_communities_list = community_counts[community_counts['Node Count'] < threshold]['Community'].tolist()

filtered_communities_df = communities_df[~communities_df['Community'].isin(small_communities_list)]

print("Communities below threshold (to be deleted):", small_communities_list)
print("\nFiltered communities_df (without nodes in small communities):")
print(filtered_communities_df)


Communities below threshold (to be deleted): [69, 65, 62, 73, 68, 71, 76, 72, 74, 77, 78, 81, 70, 80, 90, 75, 84, 88, 82, 89, 79, 86, 83, 93, 85, 87, 91, 98, 94, 95, 97, 96, 99, 115, 100, 92, 101, 102, 103, 104, 371, 112, 111, 116, 109, 105, 108, 113, 107, 153, 106, 117, 121, 122, 118, 119, 120, 136, 139, 137, 138, 134, 140, 141, 145, 147, 135, 125, 133, 131, 130, 129, 132, 110, 124, 123, 114, 150, 213, 178, 165, 156, 154, 152, 191, 148, 146, 144, 149, 143, 151, 179, 173, 174, 176, 224, 182, 221, 171, 183, 185, 187, 302, 172, 175, 170, 160, 168, 169, 164, 163, 155, 159, 211, 208, 205, 207, 206, 204, 203, 202, 394, 261, 214, 247, 215, 219, 268, 222, 232, 223, 228, 278, 201, 200, 199, 186, 158, 162, 198, 142, 167, 197, 196, 195, 194, 193, 297, 271, 269, 389, 267, 264, 258, 273, 257, 256, 255, 254, 253, 252, 251, 392, 272, 383, 274, 296, 294, 1289, 293, 299, 305, 292, 287, 284, 307, 281, 312, 323, 249, 276, 275, 250, 225, 248, 237, 220, 127, 177, 184, 226, 216, 212, 230, 233, 209, 235, 10

In [ ]:
overlapping_nodes = communities_df.groupby('Node').filter(lambda x: len(x) > 1)
print("Nodes that belong to multiple communities:")
print(overlapping_nodes[['Node', 'Community']])


Nodes that belong to multiple communities:
Empty DataFrame
Columns: [Node, Community]
Index: []


In [ ]:
community_node_counts = filtered_communities_df['Community'].value_counts().reset_index()
community_node_counts.columns = ['Community', 'Node Count']
community_node_counts.head()

,Community,Node Count
0,1,150141
1,2,44064
2,3,12443
3,4,10139
4,5,8115


In [ ]:
print(filtered_communities_df)

          Node  Community
0           88          1
1       215324          1
2       370434          1
3       370435          1
4       171356          1
...        ...        ...
272096  163746         67
272097   55714         67
272098  119590         67
272099  111315         67
272100  248572         67

[272008 rows x 2 columns]


In [ ]:
unique_nodes_count = filtered_communities_df['Node'].nunique()

print("Number of unique nodes:", unique_nodes_count)

Number of unique nodes: 272008


In [ ]:
influencer_scores_df = pd.DataFrame(list(inf_score.items()), columns=['Node', 'Influence Score'])

final_df = filtered_communities_df.merge(influencer_scores_df, on='Node', how='left')
print(final_df)

          Node  Community  Influence Score
0           88          1         143270.0
1       215324          1              NaN
2       370434          1              4.0
3       370435          1              4.0
4       171356          1              NaN
...        ...        ...              ...
272003  163746         67              NaN
272004   55714         67              3.0
272005  119590         67              8.0
272006  111315         67              NaN
272007  248572         67              NaN

[272008 rows x 3 columns]


In [ ]:
print(final_df.isna().sum())

Node                    0
Community               0
Influence Score    227960
dtype: int64


In [ ]:
df_filled =final_df.fillna(0)

In [ ]:
G3 = nx.DiGraph()

for _, row in df.iterrows():
    if not pd.isna(row['Timestamp']):
        G3.add_edge(row['Node2'], row['Node1'], timestamp=row['Timestamp'])

def get_number_of_edges(graph, node):
    in_degree = graph.in_degree(node)
    out_degree = graph.out_degree(node)
    total_edges = in_degree + out_degree

    print(f"\nNode {node} has {in_degree} incoming edges and {out_degree} outgoing edges (Total: {total_edges})")
    print(f"Outgoing edges with timestamps:")
    for _, target, data in graph.out_edges(node, data=True):
        timestamp = data.get('timestamp', 'No timestamp available')
        print(f"Edge from {node} to {target} with timestamp: {timestamp}")

node_to_check = 88
get_number_of_edges(G3, node_to_check)


Streaming output truncated to the last 5000 lines.
Edge from 88 to 47868 with timestamp: 1341580147
Edge from 88 to 36181 with timestamp: 1341407322
Edge from 88 to 224051 with timestamp: 1341407323
Edge from 88 to 330014 with timestamp: 1341407324
Edge from 88 to 297272 with timestamp: 1341407349
Edge from 88 to 816 with timestamp: 1341411414
Edge from 88 to 126776 with timestamp: 1341407360
Edge from 88 to 197712 with timestamp: 1341407362
Edge from 88 to 251041 with timestamp: 1341407363
Edge from 88 to 46560 with timestamp: 1341407365
Edge from 88 to 442868 with timestamp: 1341407371
Edge from 88 to 240611 with timestamp: 1341407373
Edge from 88 to 177053 with timestamp: 1341407379
Edge from 88 to 18534 with timestamp: 1341407379
Edge from 88 to 68832 with timestamp: 1341407383
Edge from 88 to 411221 with timestamp: 1341407384
Edge from 88 to 315915 with timestamp: 1341407393
Edge from 88 to 30261 with timestamp: 1341407396
Edge from 88 to 275764 with timestamp: 1341407396
Edge fro

In [ ]:
df_filled.head()

,Node,Community,Influence Score
0,88,1,143270.0
1,215324,1,0.0
2,370434,1,4.0
3,370435,1,4.0
4,171356,1,0.0


In [ ]:
def track_propagation_within_community(G, communities_df, df, community_id, result_df):
    """
    Tracks the propagation within a specific community and appends the average propagation time
    to a DataFrame.

    Args:
    - G: A network graph with nodes and edges.
    - communities_df: DataFrame containing 'Community', 'Node', and 'Influence Score'.
    - df: DataFrame containing 'Node2' and 'Timestamp'.
    - community_id: The ID of the community to analyze.
    - result_df: DataFrame to store the results (Community ID and Average Propagation Time).

    Returns:
    - None: Updates the result_df in place.
    """

    group = communities_df[communities_df['Community'] == community_id]
    if group.empty:
        print(f"Community {community_id} not found.")
        return

    top_influencers = group.nlargest(5, 'Influence Score')['Node'].tolist()
    print(f"Starting nodes (Top 5 influencers) for Community {community_id}: {top_influencers}")

    reach_times = {node: float('inf') for node in group['Node']}
    priority_queue = []

    propagation_tracking = defaultdict(list)

    for node in top_influencers:
        initial_time = df[(df['Node2'] == node) & (df['Node2'].isin(group['Node']))]['Timestamp'].min()

        if pd.isna(initial_time):
            print(f"Warning: Initial influencer {node} has no valid timestamp.")
            continue

        reach_times[node] = initial_time
        heapq.heappush(priority_queue, (initial_time, node))

    while priority_queue:
        current_time, current_node = heapq.heappop(priority_queue)

        if current_time > reach_times[current_node]:
            continue

        for neighbor in G.neighbors(current_node):
            if neighbor in group['Node'].values:
                propagation_tracking[current_node].append(neighbor)

                try:
                    edge_time = G.edges[current_node, neighbor].get('timestamp', float('inf'))

                    if edge_time == float('inf'):
                        print(f"Warning: Missing timestamp for edge ({current_node}, {neighbor}).")
                        continue

                    if edge_time > current_time and edge_time < reach_times[neighbor]:
                        reach_times[neighbor] = edge_time
                        heapq.heappush(priority_queue, (edge_time, neighbor))
                except KeyError:
                    print(f"Edge ({current_node}, {neighbor}) not found, skipping...")
                    continue

    total_time = sum(time for time in reach_times.values() if time != float('inf'))
    num_reachable_nodes = sum(1 for time in reach_times.values() if time != float('inf'))
    average_propagation_time = total_time / num_reachable_nodes if num_reachable_nodes > 0 else 0

    print(f"Average rumor propagation time in Community {community_id}: {average_propagation_time}")

    # Append the result to the DataFrame
    result_df.loc[len(result_df)] = {'Community': community_id, 'Average Propagation Time': average_propagation_time}


In [ ]:
result_df = pd.DataFrame(columns=['Community', 'Average Propagation Time'])

In [ ]:
def propagate_for_all_communities(G, communities_df, df):
    # Get all unique community IDs
    unique_communities = communities_df['Community'].unique()

    # Loop through each community and apply the propagation function
    for community_id in unique_communities:
        print(f"\n--- Processing Community {community_id} ---")
        track_propagation_within_community(G3, df_filled, df, community_id, result_df)

# Example usage
propagate_for_all_communities(G3, df_filled, df)


--- Processing Community 1 ---
Starting nodes (Top 5 influencers) for Community 1: [88, 677, 1988, 14454, 349]
Average rumor propagation time in Community 1: 1341417494.333004

--- Processing Community 2 ---
Starting nodes (Top 5 influencers) for Community 2: [7533, 6948, 19913, 14572, 6940]
Average rumor propagation time in Community 2: 1341420409.8462133

--- Processing Community 3 ---
Starting nodes (Top 5 influencers) for Community 3: [283, 245, 236, 96622, 3235]
Average rumor propagation time in Community 3: 1341481632.2544825

--- Processing Community 4 ---
Starting nodes (Top 5 influencers) for Community 4: [16460, 6971, 69417, 16452, 85042]
Average rumor propagation time in Community 4: 1341457434.6700745

--- Processing Community 5 ---
Starting nodes (Top 5 influencers) for Community 5: [68278, 75369, 198048, 95905, 68285]
Average rumor propagation time in Community 5: 1341423332.307054

--- Processing Community 6 ---
Starting nodes (Top 5 influencers) for Community 6: [12281

In [ ]:
def get_smallest_timestamp_for_top_influencers(communities_df, df):
    """
    Find the smallest timestamp for the top 5 influencers in each community.

    Args:
    - communities_df (DataFrame): A DataFrame containing 'Community', 'Node', and 'Influence Score'.
    - df (DataFrame): A DataFrame containing 'Node2' and 'Timestamp' columns.

    Returns:
    - DataFrame: A DataFrame with each community and the smallest timestamp of its top 5 influencers.
    """
    results = []

    # Group communities_df by 'Community'
    grouped = communities_df.groupby('Community')

    for community_id, group in grouped:
        # Get top 5 influencers based on 'Influence Score'
        top_5_influencers = group.nlargest(5, 'Influence Score')['Node']

        # Filter df for the top influencers
        filtered_df = df[df['Node2'].isin(top_5_influencers)]

        # Get the smallest timestamp for the top 5 influencers
        smallest_timestamp = filtered_df.groupby('Node2')['Timestamp'].min().min()

        results.append({'Community': community_id, 'Smallest Timestamp for Top 5 Influencers': smallest_timestamp})

    # Create a DataFrame from the results
    final_df = pd.DataFrame(results)

    return final_df

# Example usage
smallest_timestamps = get_smallest_timestamp_for_top_influencers(df_filled, df)
print(smallest_timestamps)


    Community  Smallest Timestamp for Top 5 Influencers
0           1                                1341106079
1           2                                1341171942
2           3                                1341262249
3           4                                1341172039
4           5                                1341168986
..        ...                                       ...
60         61                                1341247826
61         63                                1341431033
62         64                                1341305675
63         66                                1341385999
64         67                                1341326579

[65 rows x 2 columns]


In [ ]:
def adjust_and_sort_propagation_times(result_df, smallest_timestamps):
    """
    Adjusts the average propagation times by subtracting the smallest timestamps for top influencers
    and sorts the results by adjusted propagation time.

    Args:
    - result_df (DataFrame): DataFrame containing 'Community' and 'Average Propagation Time'.
    - smallest_timestamps (DataFrame): DataFrame containing 'Community' and 'Smallest Timestamp for Top 5 Influencers'.

    Returns:
    - DataFrame: Adjusted and sorted propagation times for each community.
    """
    # Merge the two DataFrames on the 'Community' column
    merged_df = result_df.merge(smallest_timestamps, on='Community')

    # Subtract the smallest timestamp from the average propagation time
    merged_df['Adjusted Propagation Time'] = (
        merged_df['Average Propagation Time'] - merged_df['Smallest Timestamp for Top 5 Influencers']
    )

    # Keep only the necessary columns
    adjusted_df = merged_df[['Community', 'Adjusted Propagation Time']]

    # Sort by the adjusted propagation time
    sorted_df = adjusted_df.sort_values(by='Adjusted Propagation Time', ascending=True).reset_index(drop=True)

    return sorted_df

# Example usage
sorted_result_df = adjust_and_sort_propagation_times(result_df, smallest_timestamps)
print(sorted_result_df)


    Community  Adjusted Propagation Time
0          31               17566.439394
1          49               30122.587301
2          26               30837.030055
3          47               31628.163043
4          33               33733.976048
..        ...                        ...
60         18              303435.684685
61         20              305097.891418
62          1              311415.333004
63         27              328039.504808
64         30              350086.886364

[65 rows x 2 columns]


In [ ]:
def print_community_order(result_df, smallest_timestamps):
    """
    Adjusts propagation times, sorts them, and prints the order of communities.

    Args:
    - result_df (DataFrame): DataFrame containing 'Community' and 'Average Propagation Time'.
    - smallest_timestamps (DataFrame): DataFrame containing 'Community' and 'Smallest Timestamp for Top 5 Influencers'.

    Returns:
    - List[int]: Sorted list of community IDs based on adjusted propagation time.
    """
    # Adjust and sort the DataFrame
    sorted_df = adjust_and_sort_propagation_times(result_df, smallest_timestamps)

    # Extract the sorted list of community IDs
    community_order = sorted_df['Community'].tolist()

    # Print the order
    print("Order of Communities Based on Adjusted Propagation Time:")
    print(community_order)

    return community_order

# Example usage
community_order = print_community_order(result_df, smallest_timestamps)


Order of Communities Based on Adjusted Propagation Time:
[31, 49, 26, 47, 33, 63, 24, 60, 42, 66, 51, 43, 53, 44, 48, 22, 64, 15, 61, 46, 25, 57, 58, 40, 23, 16, 32, 37, 28, 36, 34, 52, 21, 10, 56, 54, 11, 12, 59, 55, 67, 17, 8, 9, 3, 41, 14, 35, 7, 38, 2, 19, 5, 50, 29, 4, 13, 6, 39, 45, 18, 20, 1, 27, 30]


In [ ]:
def get_fastest_and_slowest_communities(result_df, smallest_timestamps):
    """
    Adjusts propagation times, sorts them, and retrieves the fastest and slowest 5 communities.

    Args:
    - result_df (DataFrame): DataFrame containing 'Community' and 'Average Propagation Time'.
    - smallest_timestamps (DataFrame): DataFrame containing 'Community' and 'Smallest Timestamp for Top 5 Influencers'.

    Returns:
    - Tuple[List[int], List[int]]: Two lists containing the fastest and slowest 5 communities.
    """
    # Adjust and sort the DataFrame
    sorted_df = adjust_and_sort_propagation_times(result_df, smallest_timestamps)

    # Get the fastest 5 communities
    fastest_communities = sorted_df.head(5)['Community'].tolist()

    # Get the slowest 5 communities
    slowest_communities = sorted_df.tail(5)['Community'].tolist()

    print("Fastest 5 Communities Based on Adjusted Propagation Time:")
    print(fastest_communities)

    print("Slowest 5 Communities Based on Adjusted Propagation Time:")
    print(slowest_communities)

    return fastest_communities, slowest_communities

# Example usage
fastest_communities, slowest_communities = get_fastest_and_slowest_communities(result_df, smallest_timestamps)


Fastest 5 Communities Based on Adjusted Propagation Time:
[31, 49, 26, 47, 33]
Slowest 5 Communities Based on Adjusted Propagation Time:
[18, 20, 1, 27, 30]


Tests

In [ ]:
def modularity(graph, communities_df):
    """
    Calculate modularity for the given graph and communities.

    Parameters:
        graph (nx.Graph): The input graph.
        communities_df (pd.DataFrame): DataFrame with columns ["Node", "Community"].

    Returns:
        float: Modularity value.
    """
    m = graph.size(weight='weight')
    community_groups = communities_df.groupby("Community")

    modularity_score = 0

    for community, nodes_df in community_groups:
        nodes = nodes_df["Node"].tolist()
        subgraph = graph.subgraph(nodes)
        e_c = subgraph.size(weight='weight')
        k_c = sum(dict(graph.degree(nodes, weight='weight')).values())
        modularity_score += (e_c / m) - ((k_c / (2 * m)) ** 2)

    return modularity_score


In [ ]:
def density(graph, communities_df):
    """
    Calculate density for each community in the graph.

    Parameters:
        graph (nx.Graph): The input graph.
        communities_df (pd.DataFrame): DataFrame with columns ["Node", "Community"].

    Returns:
        dict: A dictionary with community IDs as keys and density values as values.
    """
    density_scores = {}
    community_groups = communities_df.groupby("Community")

    for community, nodes_df in community_groups:
        nodes = nodes_df["Node"].tolist()
        subgraph = graph.subgraph(nodes)
        num_nodes = len(subgraph.nodes)
        num_edges = subgraph.size(weight='weight')

        density_scores[community] = (2 * num_edges) / (num_nodes * (num_nodes - 1)) if num_nodes > 1 else 0

    return density_scores

In [ ]:
def conductance(graph, communities_df):
    """
    Calculate conductance for each community in the graph.

    Parameters:
        graph (nx.Graph): The input graph.
        communities_df (pd.DataFrame): DataFrame with columns ["Node", "Community"].

    Returns:
        dict: A dictionary with community IDs as keys and conductance values as values.
    """
    conductance_scores = {}
    community_groups = communities_df.groupby("Community")

    for community, nodes_df in community_groups:
        nodes = nodes_df["Node"].tolist()
        subgraph = graph.subgraph(nodes)
        internal_edges = subgraph.size(weight='weight')

        # External edges: sum weights of edges leaving the community
        external_edges = 0
        for node in nodes:
            for neighbor in graph.neighbors(node):
                if neighbor not in nodes:
                    external_edges += graph[node][neighbor].get('weight', 1)

        total_edges = internal_edges + external_edges
        conductance_scores[community] = external_edges / total_edges if total_edges > 0 else 0

    return conductance_scores

In [ ]:
mod_score = modularity(G3, df_filled)
print(f"Modularity: {mod_score}")

Modularity: 0.5649853879722347


In [ ]:
density_scores = density(G3, df_filled)
print("Density Scores:")
for community, score in density_scores.items():
    print(f"Community {community}: {score}")

Density Scores:
Community 1: 2.175691185262098e-05
Community 2: 7.290615003924779e-05
Community 3: 0.00021687708036736248
Community 4: 0.0002367369749085074
Community 5: 0.00029138078742673524
Community 6: 0.000379025970362406
Community 7: 0.00041620475769196835
Community 8: 0.0006865593275114997
Community 9: 0.0008093249508624137
Community 10: 0.0009327247856976996
Community 11: 0.0009135858117776228
Community 12: 0.001247553645677353
Community 13: 0.001775512635248173
Community 14: 0.0022531456769750726
Community 15: 0.0017766786059468986
Community 16: 0.0030209382270217715
Community 17: 0.004850286254707581
Community 18: 0.0028441191081048384
Community 19: 0.0031462162609703593
Community 20: 0.0027720623148755285
Community 21: 0.003078385927378618
Community 22: 0.004566918510523476
Community 23: 0.005377868267999966
Community 24: 0.0032578259179244402
Community 25: 0.006089945296993755
Community 26: 0.004939004642124581
Community 27: 0.008187693823725473
Community 28: 0.004749938408

In [ ]:
conductance_scores = conductance(G3, df_filled)
print("Conductance Scores:")
for community, score in conductance_scores.items():
    print(f"Community {community}: {score}")

Conductance Scores:
Community 1: 0.06060281025421002
Community 2: 0.051284800879321205
Community 3: 0.017441179913379373
Community 4: 0.08256673201628714
Community 5: 0.013573264781491002
Community 6: 0.031746031746031744
Community 7: 0.04506674696376593
Community 8: 0.08350682396483923
Community 9: 0.12916111850865514
Community 10: 0.06867998051631759
Community 11: 0.02338611449451888
Community 12: 0.05099337748344371
Community 13: 0.08732694355697551
Community 14: 0.024968789013732832
Community 15: 0.01693889897156685
Community 16: 0.18590143480973176
Community 17: 0.060209424083769635
Community 18: 0.08602150537634409
Community 19: 0.07835820895522388
Community 20: 0.02929532858273951
Community 21: 0.16541353383458646
Community 22: 0.08389715832205684
Community 23: 0.08879184861717612
Community 24: 0.004511278195488722
Community 25: 0.11032531824611033
Community 26: 0.09703947368421052
Community 27: 0.02586206896551724
Community 28: 0.16464471403812825
Community 29: 0.12588652482269